In [ ]:
import pandas as pd
import numpy as np
ano_atual = 2025
semestre_atual = int(1)

pd.set_option('future.no_silent_downcasting', True)

# substituir aqui o link da planilha a ser analisada
file_path = "https://docs.google.com/spreadsheets/d/1cpXhYwbhTlIexWjTprVITGAJA8jXMU83/export?format=xlsx"
file_path_crite = "https://docs.google.com/spreadsheets/d/1fmyyHKp38u5CM0G3GYa-gbPbvokt4_SZ/export?format=xlsx"
file_paht_form = "https://docs.google.com/spreadsheets/d/1HINITMZMllcojXwgq8gOwt_P6dK6MnOe/export?format=xlsx"

# Função para padronizar e converter a coluna GRR para inteiro
def padronizar_grr(df):
    # Localiza a coluna que contém 'grr' no nome (insensível a maiúsculas)


    # Garante que a coluna exista
    if "GRR" not in df.columns:
        raise ValueError("Nenhuma coluna contendo 'GRR' foi encontrada no DataFrame.")

    # Limpa e padroniza os valores
    df["GRR"] = (
        df["GRR"]
        .astype(str)
        .str.extract(r"(\d+)")   # extrai apenas os números
        .dropna()                # remove valores nulos
        [0]                      # pega a primeira coluna do extract
        .astype(int)             # garante que é numérico
        .astype(str)             # converte novamente para string
        .radd("GRR")             # adiciona o prefixo 'GRR' à esquerda
    )

    return df


# Aplicando a padronização


############################ COLE NO FINAL DO LINK DEPOIS DA ULTIMA BARRA:  export?format=xlsx
#file_path_com_parecer = "https://docs.google.com/spreadsheets/d/1mdU9VRlCjQc-GK4sh_O7KnQu1KWjn1VE/export?format=xlsx"
############################

semestre_atual = 1 if semestre_atual < 1 else (2 if semestre_atual > 2 else semestre_atual)
xls = pd.ExcelFile(file_path)
xls_social = pd.ExcelFile(file_path_crite)
xls_psicologia = pd.ExcelFile(file_path_crite)
xl_pedagogia = pd.ExcelFile(file_path_crite)
xl_form = pd.ExcelFile(file_paht_form)
#xls_parecer = pd.ExcelFile(file_path_com_parecer)

# Mostrar as abas existentes para entender a estrutura
xls.sheet_names
xls_social.sheet_names
xls_psicologia.sheet_names
xl_pedagogia.sheet_names
xl_form.sheet_names
#xls_parecer.sheet_names
# Carregar a aba identificada
df = pd.read_excel(file_path, sheet_name="PLANILHA COMPLETA")

df_social = pd.read_excel(file_path_crite, sheet_name="Serviço Social")
df_psicologia = pd.read_excel(file_path_crite, sheet_name="Psicologia")
df_pedagogia = pd.read_excel(file_path_crite, sheet_name="pedagogia")
#df_form_ = pd.read_excel(file_paht_form, sheet_name="Sheet1")
#df_form = padronizar_grr(df_form_)
#df_parecer = pd.read_excel(file_path_com_parecer, sheet_name="pedagogia")



# Exibir todas as colunas
pd.set_option('display.max_columns', None)

#display(df.head(2))
# ========================================================
# 1. Função de normalização genérica
#    Garante que a série seja reescalada para o intervalo [0,1]
# ========================================================
def normalize(series: pd.Series) -> pd.Series:
    s = series.astype(float).fillna(0.0)
    rng = s.max() - s.min()
    return (s - s.min()) / (rng + 1e-9)



# ========================================================
# 2. Risco Aprovação
#    Se aprovação = 100% → risco = 0
#    Se aprovação = 0%   → risco = 1
#    Se entre 99% e 1% → risco = 1-(porcentagem-aprovacao/100), ou seja o risco_aprovação é o quanto falta para atingir 100%.
#    exemplo: porcentagem-aprovacao=25% => risco = 0.75
# ========================================================
# 2. Calcular o risco base
df["porcentagem_aprov_num"] =( pd.to_numeric(
    df["porcentagem-aprovacao"].replace(['#REF!', 'x'], np.nan),
    errors="coerce"
).fillna(0.0))
#df["risco_aprovacao"] = 1 - (df["porcentagem_aprov_num"] / 100.0)

# Calcular o risco de retenção (com base no rendimento)
# Fórmula: risco = ((100 - rendimento) / 100) ** 2
df["risco_aprovacao"] = (((100 - df["porcentagem_aprov_num"]) / 100) ** 2)#.round(2)

# ========================================================
# 3. Risco Cancelamento
#    Calculado a partir da fração de disciplinas canceladas
#   fração de disciplinas canceladas = ('qtd-matricula-cancelada'/ 'qtd-matriculada')
#    Se fração de disciplinas canceladas for maior igual (>=) 0.5  → risco = 1
#    Caso contrário      → risco = valor proporcional variando (0 a 0.9)
# ========================================================
# 2- porcentagem_disciplinas_canceladas foi calculada pegando 'qtd-matricula-cancelada' / 'qtd-matricula'
df["fracao_disciplinas_canceladas"]=( df['qtd-matricula-cancelada'].replace(['#REF!', 'x'], np.nan).astype(float)/df['qtd-matriculada'].replace(['#REF!', 'x'], np.nan).astype(float)).round(2)

df["risco_cancelamento"] = (np.where(
    df["fracao_disciplinas_canceladas"] >= 0.5,
    1.0,  # risco máximo
    df["fracao_disciplinas_canceladas"].clip(lower=0)  # proporcional até 0.5
)).round(2)

# ========================================================
# 4. Risco Reprovação por Frequência (qtd atual)
#    Regras fixas: se "qtd-matriculada" menor ou igual a 2 e "qtd-rep-frequencia" maior que zero  → risco alto → 1, ou
#    Se "qtd-matriculada" maior ou igual a 5 e "qtd-rep-frequencia" maior que 2 → risco alto → 1
#    para os valores que não se enquadram na regra risco reprovação por frequencia igual a zero → 0
# ========================================================
df["qtd-rep-frequencia"] = (pd.to_numeric(
    df["qtd-rep-frequencia"].replace(['#REF!', 'x'], np.nan), errors="coerce"
)).round(2)
df["qtd-matriculada"] = (pd.to_numeric(
    df["qtd-matriculada"].replace(['#REF!', 'x'], np.nan), errors="coerce"
)).round(2)

df["risco_rep_freq"] = (np.where(
    ((df["qtd-matriculada"] <= 2) & (df["qtd-rep-frequencia"] > 0)) |
    ((df["qtd-matriculada"] == 3) & (df["qtd-rep-frequencia"] > 1)) |
    ((df["qtd-matriculada"] == 4) & (df["qtd-rep-frequencia"] > 1)) |
    ((df["qtd-matriculada"] >= 5) & (df["qtd-rep-frequencia"] > 2)),
    1, 0
)).round(2)

# ========================================================
# 5. Risco Histórico de Reprovação por Frequência
#    é o Percentual de reprovação por frequência nos últimos 3 semestres Normalizado entre 0 e 1
# ========================================================
df["porcentagem-historica-de-reprovacao-frequencia"]=df["porcentagem-historica-de-reprovacao-frequencia"]
df["hist_freq_pct"] =( pd.to_numeric(
    df["porcentagem-historica-de-reprovacao-frequencia"].replace(['#REF!', 'x'], np.nan),
    errors="coerce"
).fillna(0.0))

df["risco_hist_freq"] = (df["hist_freq_pct"].clip(0, 1))

# ========================================================
# 6. Risco Carga Horária Integralizada
#    primeiro Calcula tempo de curso "semestres_cursados"
#       "semestres_cursados" = (((ano_atual - "ano-ingresso") * 2)  vezes 2 é pra chegar nos semestre
#    segundo Calcula carga horária esperada pelo tempo de curso
#       "Carga horária esperada" = "semestres_cursados" * 8
#    terceiro Calcula Risco = "ch-integralizada" / ch_media_esperada, o risco é o déficit em relação ao esperado
#    Normalizado entre 0 e 1
# ========================================================
# Converter ano e semestre de ingresso para numérico
ano_ingresso = pd.to_numeric(
    df["ano-ingresso"].replace(['#REF!', 'x'], np.nan), errors="coerce"
)
#semestre_ingresso = pd.to_numeric(
#    df["semestre-ingresso"].replace(['#REF!', 'x'], np.nan), errors="coerce"
#)

# Calcular semestres cursados do ponto (ano_ingresso/semestre_ingresso)
# até (ano_atual/semestre_atual), contando o semestre de ingresso (+1).
# Fórmula canônica:
# semestres_cursados = (ano_atual - ano_ingresso)*2 + (semestre_atual - semestre_ingresso) + 1
#df["semestres_cursados"] = (
#    (ano_atual - ano_ingresso) * 2
#    + (semestre_atual - semestre_ingresso)
#    + 1
#)

# Garantir mínimo de 1 e tratar faltas/erros
# 1) Garantias básicas
df["TEMPO UFPR - SEM"] = pd.to_numeric(df["TEMPO UFPR - SEM"], errors="coerce").fillna(1).clip(lower=1)
df["ch-integralizada"] = pd.to_numeric(df["ch-integralizada"], errors="coerce").fillna(0).clip(0, 100)

# 2) Esperado em % (NÃO clipar em 100 para permitir >100 e identificar atraso)
df["ch_media_esperada"] = 8 * df["TEMPO UFPR - SEM"]

# 3) Ajuste de tempo (mais forte para quem tem esperado > 100)
cond_boost = (df["ch_media_esperada"] > 110) & (df["porcentagem_aprov_num"] < 50)
df["ajuste_tempo"] = np.where(cond_boost, 5 * df["TEMPO UFPR - SEM"], df["TEMPO UFPR - SEM"])

# 4) Risco base (sem normalização)
df["risco_ch_integralizada"] = (
    1 - (df["ch-integralizada"] / df["ch_media_esperada"])
) * df["ajuste_tempo"]

# Sem risco negativo
df["risco_ch_integralizada"] = df["risco_ch_integralizada"].clip(lower=0)

# 5) Normalização:
#    - normalize SOMENTE quem NÃO está no boost, para preservar a separação
mask_non = ~cond_boost
if mask_non.any():
    min_val = df.loc[mask_non, "risco_ch_integralizada"].quantile(0.05)
    max_val = df.loc[mask_non, "risco_ch_integralizada"].quantile(0.95)
    span = (max_val - min_val) if (max_val - min_val) != 0 else 1.0

    df.loc[mask_non, "risco_ch_integralizada"] = (
        (df.loc[mask_non, "risco_ch_integralizada"] - min_val) / span
    ).clip(0, 1)

# 6) Isolar líderes:
#    - quem tem ch_media_esperada > 100 vai para 1.00
#    - opcional: comprimir os demais até 0.90 para garantir “folga” no topo
df.loc[cond_boost, "risco_ch_integralizada"] = 1.00
df.loc[mask_non, "risco_ch_integralizada"] = (df.loc[mask_non, "risco_ch_integralizada"] * 0.25)

# 7) Duas casas decimais
df["risco_ch_integralizada"] = df["risco_ch_integralizada"].round(2)
# ========================================================
# 7. Risco Histórico de Avaliações
#    Se já apareceu em pelo menos 1 das avaliações anteriores → risco = 1
#    se não aparece em nenhuma avaliação anterior  → risco = 0
# ========================================================
df["risco_historico"] = (np.where(
    (df["apareceu-na-avaliacao-semestre-anterior?"] == 1) |
    (df["apareceu-na-avaliacao-semestre-anterior?"] == "0"),
    1.0, 0.0
)).round(2)

# ========================================================
# 8. Risco Carga Horária cursada
#    - Se >=300h → risco = 0 (sem risco)
#    - 200–299h  → risco = 0.33
#    - 100–199h  → risco = 0.66
#    - 0–99h     → risco = 1.0
# ========================================================
df["ch_cursada"] = pd.to_numeric(
    df["TEMPO UFPR - SEM"].replace(['#REF!', 'x'], np.nan),
    errors="coerce"
).fillna(0)

df["risco_ch_cursada"] = (np.select(
    [
        df["ch_cursada"] >= 300,
        (df["ch_cursada"] <= 299) & (df["ch_cursada"] >= 200),
        (df["ch_cursada"] <= 199) & (df["ch_cursada"] >= 100),
        (df["ch_cursada"] <= 99) & (df["ch_cursada"] >= 0)
    ],
    [0, 0.33, 0.66, 1.0],   # valores normalizados
    default=np.nan
)).round(2)

# ========================================================
# 9. Pesos e Nota Final
#    Cada risco já está normalizado em [0,1].
#    A nota parcial = risco * peso (risco vezes peso)
#    Nota final = soma das notas parciais * 100 (para escala 0–100)
# ========================================================
pesos = {
    "aprovacao": 4,
    #"cancelamento": 2,
    "rep_freq": 1.5,
    "hist_freq": 0.5,
    "ch_integralizada": 3,
    "historico": 0.5,
    "ch_cursada": 0.5,
}

for chave, peso in pesos.items():
    df[f"peso_{chave}"] = peso
    df[f"nota_parcial_{chave}"] = (df[f"risco_{chave}"] * peso).round(2)

# Nota final já ajustada para ficar entre 0 e 100
df["nota_final"] = (df[[f"nota_parcial_{k}" for k in pesos.keys()]].sum(axis=1) ).round(2)

# ========================================================
# 10. Ordenação de colunas e exibição
# ========================================================
colunas_id = ["GRR", "NOME","SETOR","proafe", "curso", "ano-ingresso" ,"TEMPO UFPR - SEM","IRA SEM","CPF","renda-per-capta","classe-da-renda","nota-da-renda","E-MAIL PESSOAL","E-MAIL INSTITUCIONAL","TELEFONE","MOTIVO","planilha_andre"]
blocos = [
    (["porcentagem-aprovacao"], "risco_aprovacao", "peso_aprovacao", "nota_parcial_aprovacao"),
    #(["qtd-matricula-cancelada","qtd-matriculada","fracao_disciplinas_canceladas"], "risco_cancelamento", "peso_cancelamento", "nota_parcial_cancelamento"),
    (["qtd-matriculada","qtd-reprovacao-por-nota","qtd-matricula-cancelada","PORT 5 - CAN",
      "qtd-rep-frequencia","PORT 5 - FREQ","BAIXA MAT"], "risco_rep_freq", "peso_rep_freq", "nota_parcial_rep_freq"),
    (["porcentagem-historica-de-reprovacao-frequencia"], "risco_hist_freq", "peso_hist_freq", "nota_parcial_hist_freq"),
    (["ch-integralizada","TEMPO UFPR - SEM", "ch_media_esperada","CH REC SEM","CH ABAIXO", "CH MTO ABAIXO"], "risco_ch_integralizada", "peso_ch_integralizada", "nota_parcial_ch_integralizada"),
    (["apareceu-na-avaliacao-semestre-anterior?"], "risco_historico", "peso_historico", "nota_parcial_historico"),
    (["responsavel","TEMPO UFPR - SEM","CH MAT TOTAL","% Rep Freq 2024-2","% Rep Freq 2024-1","% Rep Freq 2023 -2","Editais 2023","AVALIAÇÃO 2024","recebeu-probem-ano-anterior?",], "risco_ch_cursada", "peso_ch_cursada", "nota_parcial_ch_cursada"),
]

ordered_cols = colunas_id.copy()
for origs, risco, peso, nota in blocos:
    ordered_cols.extend(origs + [risco, peso, nota])

# Remove duplicates while preserving order
ordered_cols = list(dict.fromkeys(ordered_cols))

ordered_cols.append("nota_final")

df_detalhado = df[ordered_cols].sort_values("nota_final", ascending=False)



df_merged = (
    df_detalhado
    .merge(df_social, on="GRR", how="left")       # junta informações sociais
    .merge(df_psicologia, on="GRR", how="left")   # junta informações de psicologia
    .merge(df_pedagogia, on="GRR", how="left")        # junta informações gerais
   # .merge(df_form, on="GRR", how="left")
  )


def aplicar_regra_renda(
    df: pd.DataFrame,
    classe_col: str = "classe-da-renda",
    nota_col: str = "nota-da-renda",
    out_col: str = "pontuacao-renda",
) -> pd.DataFrame:
    """
    Aplica a lógica da fórmula Excel:
    =SE(E(D="A";E>25);30;
      SE(E(D="A";E>=11;E<=25);25;
        SE(E(D="A";E>=1;E<=10);20;
          SE(E(D="B";E>=11);15;
            SE(E(D="B";E>=0;E<11);10;
              SE(E(D="C";E>=11);8;
                SE(E(D="C";E>=0;E<11);5;
                  SE(E(D="C";E<0);2;0)
    ))))))))
    """

    out = df.copy()

    # Normaliza tipos/valores
    classe = out[classe_col].astype(str).str.strip().str.upper()
    nota = pd.to_numeric(out[nota_col], errors="coerce")

    conditions = [
        (classe == "A") & (nota > 25),
        (classe == "A") & (nota.between(11, 25, inclusive="both")),
        (classe == "A") & (nota.between(1, 10, inclusive="both")),
        (classe == "B") & (nota >= 11),
        (classe == "B") & (nota.between(0, 10, inclusive="both")),
        (classe == "C") & (nota >= 11),
        (classe == "C") & (nota.between(0, 10, inclusive="both")),  # 0 a 10
        (classe == "C") & (nota < 0),
    ]
    choices = [30, 25, 20, 15, 10, 8, 5, 2]

    out[out_col] = np.select(conditions, choices, default=0).astype(int)
    return out
df_renda = aplicar_regra_renda(df_merged)


def _to_yesno(s: pd.Series) -> pd.Series:
    """Normaliza strings para 'SIM', 'NÃO' ou NaN (sem warnings futuros)."""
    x = s.astype(str).str.strip().str.upper()
    # Tratar 'NAO' como 'NÃO'
    x = x.str.replace(r'\bNAO\b', 'NÃO', regex=True)
    # Substituir vazios e valores nulos com compatibilidade futura
    x = x.replace({'': np.nan, 'NAN': np.nan, 'NONE': np.nan})
    # Evita o warning de downcasting
    x = x.infer_objects(copy=False)
    return x

def _to_bool_aval2024(s: pd.Series) -> pd.Series:
    """
    Converte a coluna de presença na Avaliação 2024 para booleano:
    True se esteve, False se não esteve. Valores não reconhecidos -> False.
    Aceita True/False, 1/0, 'SIM'/'NÃO'.
    """
    if s.dtype == bool:
        return s
    if np.issubdtype(s.dtype, np.number):
        return s.fillna(0).astype(int).astype(bool)
    xs = _to_yesno(s)
    return xs.map({'SIM': True, 'NÃO': False}).fillna(False)

def aplicar_indicador_acomp_adesao(
    df: pd.DataFrame,
    status_col: str = "A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não)",
    avaliacao2024_col: str = "esteve-na-avaliacao-2024",
    chi_col: str = "ch_media_esperada",
    out_score_col: str = "indicador-acomp-adesao",
    out_class_col: str = "classificacao-acomp-adesao",
) -> pd.DataFrame:
    """
    Regras implementadas:
    - Ingressante (CHI < 24%): 0,0  | 'Sem risco / não pontua'
    - NÃO esteve na Aval. 2024 & SIM: 0,2  | 'Estável'
    - NÃO esteve na Aval. 2024 & NÃO: 0,6  | 'Em alerta'
    - NÃO esteve na Aval. 2024 & em branco: 0,8  | 'Prioridade de inserção'
    - Esteve na Aval. 2024 & SIM: 0,1  | 'Estável'
    - Esteve na Aval. 2024 & NÃO: 1,0  | 'Crítico / penalização máxima'
    - Esteve na Aval. 2024 & em branco: 0,9  | 'Prioridade de convocação'
    """
    out = df.copy()

    # Coluna de status (SIM/NÃO/em branco)
    if status_col not in out.columns:
        raise KeyError(f"Coluna não encontrada: {status_col!r}")
    status = _to_yesno(out[status_col])

    # Participação na Avaliação PROBEM 2024
    if avaliacao2024_col not in out.columns:
        # Se não existir, assume False (não esteve na avaliação 2024)
        esteve2024 = pd.Series(False, index=out.index)
    else:
        esteve2024 = _to_bool_aval2024(out[avaliacao2024_col])

    # Ingressante (CHI esperada < 24%)
    if chi_col in out.columns:
        chi = pd.to_numeric(out[chi_col], errors="coerce")
        ingressante = chi < 24
    else:
        ingressante = pd.Series(False, index=out.index)

    # Condições conforme a matriz de regras
    # Ordem importa: ingressante tem precedência (neutro 0,0)
    conditions = [
        ingressante,  # 0
        (~esteve2024) & (status == "SIM"),   # 1
        (~esteve2024) & (status == "NÃO"),   # 2
        (~esteve2024) & (status.isna()),     # 3 (em branco)
        (esteve2024) & (status == "SIM"),    # 4
        (esteve2024) & (status == "NÃO"),    # 5
        (esteve2024) & (status.isna()),      # 6 (em branco)
    ]
    scores = [0.0, 0.2, 0.6, 0.8, 0.1, 1.0, 0.9]
    classes = [
        "Sem risco / não pontua",
        "Estável",
        "Em alerta",
        "Prioridade de inserção",
        "Estável",
        "Crítico / penalização máxima",
        "Prioridade de convocação",
    ]

    out[out_score_col] = np.select(conditions, scores, default=np.nan).astype(float)
    out[out_class_col] = np.select(conditions, classes, default="Regra não classificada").astype(str)
    return out

df_acomp = aplicar_indicador_acomp_adesao(df_renda)

def calcular_ita(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula o ITA ponderado e gera classificação de risco.

    Pesos:
        - nota_final → 6
        - pontuacao-renda → 3
        - indicador-acomp-adesao → 1

    Fórmula:
        ITA = ((nota_final * 6) + (pontuacao-renda * 3) + (indicador-acomp-adesao * 1)) / 10

    Classificação sugerida:
        ITA < 0.3  → 'baixo risco'
        0.3–0.6    → 'risco moderado'
        > 0.6      → 'risco alto'
    """
    df = df.copy()
    cols = ["nota_final", "pontuacao-renda", "indicador-acomp-adesao"]

    # Garantir que as colunas existam e sejam numéricas
    for col in cols:
        if col not in df.columns:
            df[col] = np.nan
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Aplica os pesos
    pesos = {"nota_final": 6, "pontuacao-renda": 3, "indicador-acomp-adesao": 1}
    soma_pesos = sum(pesos.values())

    df["ITA"] = (
        (df["nota_final"] * pesos["nota_final"]) +
        (df["pontuacao-renda"] * pesos["pontuacao-renda"]) +
        (df["indicador-acomp-adesao"] * pesos["indicador-acomp-adesao"])
    ) / soma_pesos

    # Classificação conforme faixas
    conditions = [
        df["ITA"] < 0.3,
        df["ITA"].between(0.3, 0.6, inclusive="both"),
        df["ITA"] > 0.6
    ]
    classes = ["baixo risco", "risco moderado", "risco alto"]

    df["classificacao_ita"] = np.select(conditions, classes, default="não classificado")
    df = df.sort_values(by="ITA", ascending=False).reset_index(drop=True)
    return df

df_ITA = calcular_ita(df_acomp)
display(df_ITA)

df_merged_final = (
    df_ITA
    .merge(df_form, on="GRR", how="left")
  )
display(df_merged_final)

#classe-da-renda
#nota-da-renda
#ch_media_esperada



,GRR,NOME,SETOR,proafe,curso,ano-ingresso,TEMPO UFPR - SEM,IRA SEM,CPF,renda-per-capta,classe-da-renda,nota-da-renda,E-MAIL PESSOAL,E-MAIL INSTITUCIONAL,TELEFONE,MOTIVO,planilha_andre,porcentagem-aprovacao,risco_aprovacao,peso_aprovacao,nota_parcial_aprovacao,qtd-matriculada,qtd-reprovacao-por-nota,qtd-matricula-cancelada,PORT 5 - CAN,qtd-rep-frequencia,PORT 5 - FREQ,BAIXA MAT,risco_rep_freq,peso_rep_freq,nota_parcial_rep_freq,porcentagem-historica-de-reprovacao-frequencia,risco_hist_freq,peso_hist_freq,nota_parcial_hist_freq,ch-integralizada,ch_media_esperada,CH REC SEM,CH ABAIXO,CH MTO ABAIXO,risco_ch_integralizada,peso_ch_integralizada,nota_parcial_ch_integralizada,apareceu-na-avaliacao-semestre-anterior?,risco_historico,peso_historico,nota_parcial_historico,responsavel,CH MAT TOTAL,% Rep Freq 2024-2,% Rep Freq 2024-1,% Rep Freq 2023 -2,Editais 2023,AVALIAÇÃO 2024,recebeu-probem-ano-anterior?,risco_ch_cursada,peso_ch_cursada,nota_parcial_ch_cursada,nota_final,NOME COMPLETO_x,A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não)_x,Observações (preenchimento opcional)_x,Servidor de Referência,NOME COMPLETO_y,A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não)_y,Observações (preenchimento opcional)_y,Servidor de Referência,NOME COMPLETO,A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não),Observações (preenchimento opcional),Servidor Responsável,pontuacao-renda,indicador-acomp-adesao,classificacao-acomp-adesao,ITA,classificacao_ita
0,GRR20173632,KARINE CRISTINE PAULINO DOS ANJOS,SETOR PALOTINA,NaN,Engenharia de Energia,2017,17,0.0409,11400794927,500.00,A,30,kahcpanjos@gmail.com,karine.cristine@ufpr.br,NaN,BAIXO RENDIMENTO,NaN,0.000000,1.000000,4,4.00,6,0,2,,4,ATENCAO,,1,1.5,1.5,0.402778,0.402778,0.5,0.20,69.1,136,0,ATENCAO,,1.00,3,3.00,0,0.0,0.5,0.0,RENATA C COSTA GOTARDO,255,37.500000,0,83.333333,NaN,NaN,NaN,1.0,0.5,0.5,9.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,0.8,Prioridade de inserção,14.600,risco alto
1,GRR20165187,GILSON GUMY BORGES,SETOR DE TECNOLOGIA,TEA,Engenharia Mecânica,2016,19,0.3386,7863058945,600.00,A,30,gilsonborges@outlook.com,gilsonborges@ufpr.br,4.199815e+10,BAIXO RENDIMENTO,NaN,0.000000,1.000000,4,4.00,2,0,0,,2,ATENCAO,ATENCAO,1,1.5,1.5,0.000000,0.000000,0.5,0.00,67.8,152,0,ATENCAO,ATENCAO,1.00,3,3.00,0,0.0,0.5,0.0,NaN,105,0.000000,0,0,SIM,NaN,SIM,1.0,0.5,0.5,9.00,GILSON GUMY BORGES,NaN,NaN,NaN,GILSON GUMY BORGES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,0.8,Prioridade de inserção,14.480,risco alto
2,GRR20182768,DIEUNY CASTERLING WILBERTE GALBERT,SETOR DE TECNOLOGIA,REFUGIADO,Engenharia Química,2018,14,0.2233,1385209925,150.00,A,45,casterlinggeffry@gmail.com,dieunycasterling@ufpr.br,4.199838e+10,BAIXO RENDIMENTO,NaN,14.285714,0.734694,4,2.94,7,2,0,,4,ATENCAO,,1,1.5,1.5,0.111111,0.111111,0.5,0.06,67.7,112,0,ATENCAO,,1.00,3,3.00,1,1.0,0.5,0.5,DALMO,315,33.333333,0,0,SIM,ATÉ 2 REPROVAÇÕES POR FREQUÊNCIA,SIM,1.0,0.5,0.5,8.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,0.8,Prioridade de inserção,14.180,risco alto
3,GRR20205504,MARINA RAFAELA DIAS DOS SANTOS,SETOR DE TECNOLOGIA,-,Engenharia Química,2018,14,0.2650,8335403945,700.00,A,40,marinarafaela5.4@gmail.com,marina.rafaela@ufpr.br,4.199125e+10,BAIXO RENDIMENTO,NaN,0.000000,1.000000,4,4.00,4,4,0,,0,,,0,1.5,0.0,0.466667,0.466667,0.5,0.23,52.5,112,0,ATENCAO,ATENCAO,1.00,3,3.00,1,1.0,0.5,0.5,CAROLINA,240,50.000000,40,50,SIM,10 A 20%,SIM,1.0,0.5,0.5,8.23,MARINA RAFAELA DIAS DOS SANTOS,NaN,NaN,NaN,MARINA RAFAELA DIAS DOS SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,0.8,Prioridade de inserção,14.018,risco alto
4,GRR20193394,BRUNO RANNIERY DA SILVA,SETOR DE CIÊNCIAS DA SAÚDE,DEFICIÊNCIA FÍSICA,Medicina,2019,12,0.0000,5960787490,0.10,A,55,brunoranniery@hotmail.com,brunoranniery@ufpr.br,NaN,BAIXO RENDIMENTO,SIM,0.000000,1.000000,4,4.00,7,1,0,,6,ATENCAO,,1,1.5,1.5,0.884259,0.884259,0.5,0.44,29.0,96,800,ATENCAO,ATENCAO,0.25,3,0.75,1,1.0,0.5,0.5,CAROLINA LANGNOR E SOUSA LISBOA,390,100.000000,87.5,77.777778,SIM,ATÉ 10%,SIM,1.0,0.5,0.5,7.69,BRUNO RANNIERY DA SILVA,Sim,NaN,Sandra,BRU

,GRR,NOME,SETOR,proafe,curso,ano-ingresso,TEMPO UFPR - SEM,IRA SEM,CPF_x,renda-per-capta,classe-da-renda,nota-da-renda,E-MAIL PESSOAL,E-MAIL INSTITUCIONAL,TELEFONE,MOTIVO,planilha_andre,porcentagem-aprovacao,risco_aprovacao,peso_aprovacao,nota_parcial_aprovacao,qtd-matriculada,qtd-reprovacao-por-nota,qtd-matricula-cancelada,PORT 5 - CAN,qtd-rep-frequencia,PORT 5 - FREQ,BAIXA MAT,risco_rep_freq,peso_rep_freq,nota_parcial_rep_freq,porcentagem-historica-de-reprovacao-frequencia,risco_hist_freq,peso_hist_freq,nota_parcial_hist_freq,ch-integralizada,ch_media_esperada,CH REC SEM,CH ABAIXO,CH MTO ABAIXO,risco_ch_integralizada,peso_ch_integralizada,nota_parcial_ch_integralizada,apareceu-na-avaliacao-semestre-anterior?,risco_historico,peso_historico,nota_parcial_historico,responsavel,CH MAT TOTAL,% Rep Freq 2024-2,% Rep Freq 2024-1,% Rep Freq 2023 -2,Editais 2023,AVALIAÇÃO 2024,recebeu-probem-ano-anterior?,risco_ch_cursada,peso_ch_cursada,nota_parcial_ch_cursada,nota_final,NOME COMPLETO_x,A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não)_x,Observações (preenchimento opcional)_x,Servidor de Referência,NOME COMPLETO_y,A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não)_y,Observações (preenchimento opcional)_y,Servidor de Referência,NOME COMPLETO,A/O ESTUDANTE ATENDE AOS CRITÉRIOS? (Sim ou Não),Observações (preenchimento opcional),Servidor Responsável,pontuacao-renda,indicador-acomp-adesao,classificacao-acomp-adesao,ITA,classificacao_ita,2,Hora de início,Hora de conclusão,Email,Nome,Nome completo e sem abreviações,"Nome social: \n(Decreto nº 8.727/2016, estabelece o direito ao uso do Nome Social e o reconhecimento da identidade de gênero de pessoas dentro da comunidade LGBTQIAPN+ no âmbito da administração públic",CPF_y,Qual a sua nacionalidade?,"Qual é a sua raça/cor, de acordo com a sua autodeclaração?",E-mail institucional (@ufpr),TELEFONE com DDD,Em qual Campus você estuda?,A quais auxílios PROBEM você está vinculado?,Por qual período você recebeu o auxílio permanência?,Assinale a(s) alternativas que melhor descreve(m) sua condição de saúde ou aprendizagem:\n,Anexe laudo médico se respondeu afirmativamente à questão anterior.,"Seu desempenho acadêmico no 1º semestre de 2025 foi impactado por agravantes de saúde física? Descreva detalhadamente (acidentes, doenças, internações etc.) ou clique em ""Não se aplica"".",Anexe documentos comprobatórios da situação relatada na questão XXX,"Seu desempenho acadêmico no 1º semestre de 2025 foi impactado por agravantes de saúde mental? Descreva detalhadamente (luto, problemas familiares, problemas de relacionamento interpessoal, eventos tra",Anexe documentos comprobatórios da situação relatada na questão anterior.,"Seu desempenho acadêmico no 1º semestre de 2025 foi impactado por agravantes socioeconômicos? Descreva detalhadamente (cuidados com familiar doente, cuidado com filhos, mudança de moradia, situação de",Anexe documentos comprobatórios da situação relatada na questão anterior.,"Seu desempenho acadêmico no 1º semestre de 2025 foi impactado por situações de cunho pedagógico? Descreva detalhadamente (cogitou abandono do curso, acumulou muitas atividades extracurriculares, matri",Descreva com detalhe os motivos que fizeram você ter um baixo rendimento acadêmico no 1° Semestre de 2025,Anexe documentos comprobatórios da situação relatada na questão XXX3,"Se você esteve em acompanhamento com profissional da P4E ou SEPOL durante o ano de 2025, descreva abaixo qual/quais área(s) (Pedagogia, Psicologia, Serviço Social) e o(s) nome(s) do(s) profissional(is","Se você esteve em acompanhamento com professoras/es ou coordenadoras/es pelo POA ou COA durante o ano de 2025, descreva abaixo qual/quais professoras/es te atendeu/atenderam:",Declaração: Declaro que as informações fornecidas neste formulário são verdadeiras e estou ciente de que a apresentação de documentos comprobatórios falsos ou a omissão de informações pode resultar em,Declaro conhecimento que os dados aqui relatados estão protegidos e serão di

In [ ]:
# Function to standardize and convert the GRR column to an integer
#def padronizar_grr(df):
#    for col in df.columns:
#        if "grr" in col.lower():
#            df = df.rename(columns={col: "grr"})
#            break
    # Clean and convert GRR to integer, removing non-digits
#    df["grr"] = (
#        df["grr"]
#        .astype(str)
#        .str.extract(r"(\d+)")  # extract only numbers
#        .astype(float)         # treat missing values as NaN
#        .dropna()              # remove rows without GRR
#        .astype(int)
#    )
#    return df

# Applying standardization
#df_form = padronizar_grr(df_detalhado)
#df2 = padronizar_grr(df_parecer)

In [ ]:
#df_merged = df1.merge(
#    df2[["grr", "Parecer final"]],
#    on="grr",
#    how="left"
#)

In [ ]:
#display(df_merged)

,grr,nome,curso,ano-ingresso,semestre-ingresso,semestres_cursados,porcentagem-aprovacao,risco_aprovacao,peso_aprovacao,nota_parcial_aprovacao,qtd-matricula-cancelada,qtd-matriculada,fracao_disciplinas_canceladas,risco_cancelamento,peso_cancelamento,nota_parcial_cancelamento,qtd-matriculada,qtd-rep-frequencia,risco_rep_freq,peso_rep_freq,nota_parcial_rep_freq,porcentagem-historica-de-reprovacao-frequencia,risco_hist_freq,peso_hist_freq,nota_parcial_hist_freq,ch-integralizada,semestres_cursados,ch_media_esperada,risco_ch_integralizada,peso_ch_integralizada,nota_parcial_ch_integralizada,apareceu-na-avaliacao-semestre-anteanterior?,apareceu-na-avaliacao-semestre-anterior?,risco_historico,peso_historico,nota_parcial_historico,carga-horaria-matriculada-semestre-atual,responsavel,risco_ch_atual,peso_ch_atual,nota_parcial_ch_atual,nota_final,Parecer final
0,20240120,PEDRO JAVIER LEYTON OBLITAS,DESIGN DE PRODUTO,2024,1.0,2.0,0,1.250000,0.25,0.312500,0,1.0,0.000000,0.000000,0.1,0.000000,1.0,1.0,1,0.15,0.15,10.0,0.100,0.1,0.0100,0.0,2.0,25.0,1.0000,0.25,0.25000,não,sim,1.0,0.1,0.1,150.0,DALMO,0.66,0.05,0.0330,85.550000,suspender auxílio
1,20212875,LUIZA MANUELA NATEL,ENGENHARIA QUÍMICA,2021,1.0,8.0,16.666667,1.083333,0.25,0.270833,1,7.0,0.142857,0.142857,0.1,0.014286,7.0,4.0,1,0.15,0.15,40.0,0.400,0.1,0.0400,15.3,8.0,100.0,0.8470,0.25,0.21175,sim,sim,1.0,0.1,0.1,210.0,CAROLINA,0.33,0.05,0.0165,80.336905,MANTER
2,20204517,MARCEL RODRIGUES CARDOZO,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,2020,1.0,10.0,20,1.050000,0.25,0.262500,3,8.0,0.375000,0.375000,0.1,0.037500,8.0,4.0,1,0.15,0.15,43.8,0.438,0.1,0.0438,24.7,10.0,125.0,0.8024,0.25,0.20060,sim,sim,1.0,0.1,0.1,300.0,CÍNTIA,0.00,0.05,0.0000,79.440000,Manter
3,20235150,MARIA ALANA RAMOS MARTINI,ENGENHARIA INDUSTRIAL MADEIREIRA,2023,1.0,4.0,20,1.050000,0.25,0.262500,1,6.0,0.166667,0.166667,0.1,0.016667,6.0,3.0,1,0.15,0.15,35.7,0.357,0.1,0.0357,7.4,4.0,50.0,0.8520,0.25,0.21300,sim,sim,1.0,0.1,0.1,330.0,CAROLINA,0.00,0.05,0.0000,77.786667,MANTER
4,20243345,DANIELA VERONICA PEREZ PINTO,MÚSICA,2024,1.0,2.0,28.571429,0.964286,0.25,0.241071,2,9.0,0.222222,0.222222,0.1,0.022222,9.0,4.0,1,0.15,0.15,42.9,0.429,0.1,0.0429,4.7,2.0,25.0,0.8120,0.25,0.20300,não,sim,1.0,0.1,0.1,405.0,DALMO,0.00,0.05,0.0000,75.919365,Manter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,20242771,DAN LOOGHER SANDER FILS AIME,FILOSOFIA,2024,1.0,2.0,87.5,0.187500,0.25,0.046875,0,8.0,0.000000,0.000000,0.1,0.000000,8.0,1.0,0,0.15,0.00,10.0,0.100,0.1,0.0100,37.5,2.0,25.0,0.0000,0.25,0.00000,não,sim,1.0,0.1,0.1,300.0,CAROLINA,0.00,0.05,0.0000,15.687500,MANTER
419,20214262,DIENA ALMEIDA SOUSA,TURISMO,2021,1.0,8.0,100,0.000000,0.25,0.000000,0,3.0,0.000000,0.000000,0.1,0.000000,3.0,0.0,0,0.15,0.00,10.0,0.100,0.1,0.0100,86.6,8.0,100.0,0.1340,0.25,0.03350,sim,sim,1.0,0.1,0.1,375.0,DALMO,0.00,0.05,0.0000,14.350000,Manter
420,20233498,VINICIUS DA SILVA NOGUEIRA,CIÊNCIAS CONTÁBEIS,2023,1.0,4.0,100,0.000000,0.25,0.000000,0,7.0,0.000000,0.000000,0.1,0.000000,7.0,0.0,0,0.15,0.00,10.0,0.100,0.1,0.0100,46.0,4.0,50.0,0.0800,0.25,0.02000,sim,sim,1.0,0.1,0.1,330.0,CAROLINA,0.00,0.05,0.0000,13.000000,MANTER
421,20220791,EDUARDA DE MATOS MARIA,RELAÇÕES PÚBLICAS,2022,1.0,6.0,100,0.000000,0.25,0.000000,0,12.0,0.000000,0.000000,0.1,0.000000,12.0,0.0,0,0.15,0.00,10.0,0.100,0.1,0.0100,81.9,6.0,75.0,0.0000,0.25,0.00000,sim,sim,1.0,0.1,0.1,225.0,CÍNTIA,0.33,0.05,0.0165,12.650000,Manter


In [ ]:
#PARA ABRIR E TER ACESSO O GOOGLE DRIVE
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
#PARA SALVAR O ARQUIVO COM O PADRÃO DE NOME: Analise_Probem_ano_atual_semestre_atual.xlsx
output_path = "Analise_Probem_ITA_"+str(ano_atual)+"_"+str(semestre_atual)+".xlsx"
df_merged_final.to_excel(output_path, index=False)
#df_merged.to_excel(output_path, index=False)
output_path

'Analise_Probem_ITA_2025_1.xlsx'

In [ ]:
https://docs.google.com/spreadsheets/d/1cpXhYwbhTlIexWjTprVITGAJA8jXMU83/edit?usp=sharing&ouid=107174123684363016708&rtpof=true&sd=true
https://docs.google.com/spreadsheets/d/1z8WSjHmvlaNIoGtcIHbjOi6tbjdV7J_e/edit?usp=sharing&ouid=107174123684363016708&rtpof=true&sd=true

In [ ]:
import pandas as pd
import numpy as np
DF1 = "https://docs.google.com/spreadsheets/d/1cpXhYwbhTlIexWjTprVITGAJA8jXMU83/export?format=xlsx"
DF2 = "https://docs.google.com/spreadsheets/d/1JxHSmSBmcDg_wDoiT1IHI8eydyOosUjv/export?format=xlsx"

xls = pd.ExcelFile(DF1)
xls2 = pd.ExcelFile(DF2)
df_1 = pd.read_excel(DF1, sheet_name="PLANILHA COMPLETA")
df_2 = pd.read_excel(DF2, sheet_name="COMPLETA ")


In [ ]:
df_merged_final_2 = (
    df_2
    .merge(df_1, on="GRR", how="inner")
  )
display(df_merged_final_2)

,GRR,CPF_x,NOME_x,"Nome social: \n(Decreto nº 8.727/2016, estabelece o direito ao uso do Nome Social e o reconhecimento da identidade de gênero de pessoas dentro da comunidade LGBTQIAPN+ no âmbito da administração públic",autodeclaração,Qual a sua nacionalidade?,SETOR_x,Em qual Campus você estuda?,curso_x,proafe_x,...,risco_historico_y,peso_historico_y,nota_parcial_historico_y,responsavel_y,planilha_andre_y,MOTIVO_y,E-MAIL PESSOAL_y,E-MAIL INSTITUCIONAL_y,TELEFONE_y,nota_final_rendimento
0,GRR20173084,39371304898,ANA PAULA DE OLIVEIRA FREITAS,NaN,NaN,NaN,SETOR DE CIÊNCIAS DA SAÚDE,NaN,Terapia Ocupacional,NaN,...,1,0.5,0.5,DALMO VINICIO QUEIROZ DA SILVA,SIM,BAIXO RENDIMENTO,anapaulaoliveirafreitas@yahoo.com.br,p.oliveira@ufpr.br,NaN,9.66
1,GRR20180940,5628689920,FABIAN DOMINGUES,NaN,Preta,Brasileira,SETOR DE CIÊNCIAS AGRÁRIAS,Curitiba,Medicina Veterinária,NaN,...,1,0.5,0.5,CÍNTIA,NaN,BAIXO RENDIMENTO,fabianelosta@gmail.com,fabian@ufpr.br,4.199193e+09,9.35
2,GRR20182393,5025304997,FERNANDO NILLSSON CIDADE,NaN,Branca,Brasileira,SETOR DE TECNOLOGIA,Curitiba,Engenharia Civil,NaN,...,1,0.5,0.5,CÍNTIA,NaN,BAIXO RENDIMENTO,fernando.n.cidade@outlook.com,fernando.n.cidade@ufpr.br,4.198796e+10,9.42
3,GRR20187126,6291332981,SIMONY MARTINS ROCHA MORO,NaN,Parda,Brasileira,SETOR DE CIÊNCIAS HUMANAS,Curitiba,Letras - Português,NaN,...,1,0.5,0.5,CÍNTIA,NaN,BAIXO RENDIMENTO,sedeniresimony@gmail.com,simony.moro@ufpr.br,4.199764e+10,9.56
4,GRR20173632,11400794927,KARINE CRISTINE PAULINO DOS ANJOS,NaN,Branca,Brasileira,SETOR PALOTINA,Palotina,Engenharia de Energia,NaN,...,0,0.5,0.0,RENATA C COSTA GOTARDO,NaN,BAIXO RENDIMENTO,kahcpanjos@gmail.com,karine.cristine@ufpr.br,NaN,8.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,GRR20231214,12842502965,DEBORA ANDREINA REGIS DE LIMA DA SILVA,NaN,NaN,NaN,SETOR DE EDUCAÇÃO,NaN,Pedagogia,-,...,0,0.5,0.0,NaN,NaN,BAIXA MATRÍCULA,dehboraandreinaregis@gmail.com,deboraregis@ufpr.br,4.198458e+10,0.33
468,GRR20211694,13023968977,MATHEUS MISTURINI RIEGER,NaN,NaN,NaN,SETOR DE TECNOLOGIA,NaN,Arquitetura e Urbanismo,-,...,0,0.5,0.0,NaN,NaN,BAIXA MATRÍCULA,riegermatheusrieger@gmail.com,matheus.rieger@ufpr.br,4.698832e+10,0.33
469,GRR20210565,46973712835,DARIANE CARVALHO DOS SANTOS,NaN,NaN,NaN,SETOR DE CIÊNCIAS BIOLÓGICAS,NaN,Biomedicina,-,...,0,0.5,0.0,NaN,NaN,BAIXA MATRÍCULA,daricarvalho18@gmail.com,dariane.carvalho@ufpr.br,NaN,0.50
470,GRR20220064,10076745988,DANIEL CELESTINO DE LINS NETO,NaN,NaN,NaN,SETOR DE CIÊNCIAS EXATAS,NaN,Ciência da Computação,-,...,0,0.5,0.0,NaN,NaN,BAIXA MATRÍCULA,daniellinsneto@gmail.com,daniellins@ufpr.br,4.999813e+10,0.33
